In [60]:
import os
import sys
import pandas as pd
import numpy as np

import requests
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re

import matplotlib.pyplot as plt
import matplotlib.image as matimg

from PIL import Image
from io import BytesIO
from imageio import imread

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from scipy.spatial import distance

W, H = 244, 244

%matplotlib inline
FOLDER_PATH = '/home/vahidsanei_google_com/'
PATH_DRIVER = '/home/vahidsanei_google_com/chromedriver/chromedriver'

In [2]:
df = pd.read_csv(os.path.join(FOLDER_PATH, 'data', 'yelp_data', 'updated', 'bussiness_with_corpus.csv'))
df.head()

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,...,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,is_eng,webpage_corpus
0,0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,...,0,0,0,0,1,0,0,0,True,Shooting Ranges Gun Rental Charlotte NC The Ra...
1,1,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,...,0,0,0,0,0,0,0,1,False,NaN
2,2,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,...,0,0,0,0,0,0,1,0,True,Home Renovations and Repairs Phoenix AZ Home U...
3,3,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,"1720 W Elliot Rd, Ste 105",Gilbert,AZ,85233,33.350399,-111.827142,4.5,...,0,0,0,0,0,0,1,0,True,Home Oasis Auto CenterOasis Auto Center Pages ...
4,4,ScYkbYNkDgCneBrD9vqhCQ,Junction Tire & Auto Service,6910 E Southern Ave,Mesa,AZ,85209,33.393885,-111.682226,5.0,...,0,0,0,0,0,0,0,1,True,Contact Junction Tire Tires Auto Repair Shop i...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56065 entries, 0 to 56064
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      56065 non-null  int64  
 1   business_id     56065 non-null  object 
 2   name            56065 non-null  object 
 3   address         53689 non-null  object 
 4   city            56065 non-null  object 
 5   state           56065 non-null  object 
 6   postal_code     55956 non-null  object 
 7   latitude        56065 non-null  float64
 8   longitude       56065 non-null  float64
 9   stars           56065 non-null  float64
 10  review_count    56065 non-null  int64  
 11  is_open         56065 non-null  int64  
 12  attributes      48188 non-null  object 
 13  categories      55963 non-null  object 
 14  hours           45045 non-null  object 
 15  url             56065 non-null  object 
 16  webpage_text    50471 non-null  object 
 17  1.0             56065 non-null 

In [4]:
data = df.sample(frac=0.0005)
data = data.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      28 non-null     int64  
 1   business_id     28 non-null     object 
 2   name            28 non-null     object 
 3   address         28 non-null     object 
 4   city            28 non-null     object 
 5   state           28 non-null     object 
 6   postal_code     28 non-null     object 
 7   latitude        28 non-null     float64
 8   longitude       28 non-null     float64
 9   stars           28 non-null     float64
 10  review_count    28 non-null     int64  
 11  is_open         28 non-null     int64  
 12  attributes      26 non-null     object 
 13  categories      28 non-null     object 
 14  hours           24 non-null     object 
 15  url             28 non-null     object 
 16  webpage_text    26 non-null     object 
 17  1.0             28 non-null     int64

In [5]:
data.head()

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,...,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,is_eng,webpage_corpus
0,11174,_cs_J4j8XXyBgihJs83qUg,Church's Chicken,7925 S Rainbow Blvd,Las Vegas,NV,89139,36.044787,-115.244026,1.5,...,1,0,0,0,0,0,0,0,False,NaN
1,39649,nK4ebFwSWkWs6A1TdOs8CQ,Wynn Fitness Clubs - North York,2737 Keele Street,Toronto,ON,M3M 2E9,43.726001,-79.480126,2.5,...,0,0,1,0,0,0,0,0,True,Not Acceptable Not Acceptable An appropriate r...
2,48329,4fJucVBPoEwpi5h9eHTbjg,Arby's,972 Graham Rd,Cuyahoga Falls,OH,44221,41.163167,-81.478251,2.5,...,0,0,1,0,0,0,0,0,True,About Home Grocery Supplies Rewards Community ...
3,13892,4i3sN4htL3kReuCkJbpNSQ,Smiley Crane Service,5326 W Mohave St,Phoenix,AZ,85043,33.431878,-112.174557,5.0,...,0,0,0,0,0,0,0,1,True,Smiley Crane Service Crane Rentals from 5 300 ...
4,12954,o9z_I6uLuI1bLU_GaRmdfA,Calgary Co-op,35 Crowfoot Way NW,Calgary,AB,T3G 2L4,51.123673,-114.200950,3.5,...,0,0,0,0,1,0,0,0,True,Brentwood Food Centre Gas Bar Pharmacy Calgary...


In [6]:
from urllib.parse import urlsplit

for i in range(5):
    print(data['url'][i], ':')
    split_url = urlsplit(data['url'][i])
    print(split_url.hostname)

https://locations.churchs.com/nv-las-vegas-8616 :
locations.churchs.com
http://www.voguebistro.com/ :
www.voguebistro.com
http://www.thewaggingtaillv.com :
www.thewaggingtaillv.com
http://www.smileycrane.com :
www.smileycrane.com
http://www.calgarycoop.com/stores/brentwood :
www.calgarycoop.com


In [7]:
# I found out that beautiful soup does not extract all images
c = 0
def retrieve_images_with_bs4(webpage_html):
    global c
    if webpage_html is np.nan:
        print('Nothing', '*' * 50)
        return
    soup = BeautifulSoup(webpage_html)
    for img_content in soup.find_all('img'):
        if not 'src' in str(img_content): continue
        try:
            print(img_content['src'])
            c += 1
        except Exception as e:
            print(e)
        #print(img_content)

In [8]:
# Selenium seems to be a better option

def retrieve_images(url_link, options, element, log=False):
    driver = webdriver.Chrome(PATH_DRIVER, options=options)
    try:
        driver.set_page_load_timeout(10)
        driver.implicitly_wait(10)
        driver.get(url_link)
        WebDriverWait(driver, 10).until(element)
    except:
        if log is True: print('Takes too much time.')
        return []
    images = driver.find_elements_by_tag_name('img')
    res = []
    for img in images:
        try:
            res.append(img.get_attribute('src'))
        except Exception as e:
            if log is True: print(e)
            pass
    driver.close()
    return res

In [9]:
%%time
options = Options()
options.add_argument('--headless')
element = EC.presence_of_all_elements_located((By.TAG_NAME, 'img'))
img_urls = []
for i in range(5):
    split_url = urlsplit(data['url'][i])
    images = retrieve_images('{}://'.format(split_url.scheme) + split_url.hostname, options, element)
    img_urls.extend(images)
    print('{},'.format(len(images)), end='')

6,22,5,8,44,CPU times: user 124 ms, sys: 312 ms, total: 436 ms
Wall time: 24.8 s


In [10]:
print(len(img_urls))

85


In [11]:
i = 0
for img_url in img_urls:
    try:
        r = requests.get(img_url, stream=True, timeout=3)
        file_name = 'images/pic{}.jpg'.format(i)
#         #print(img_url)
#         with open('images/pic{}.jpg'.format(i), 'wb') as f:
#             f.write(r.content)
        img = Image.open(BytesIO(r.content))
        if img.mode in ['RGBA', 'P']:
            img = img.convert('RGB')
        img.save(file_name, 'JPEG')
        
        i += 1
    except Exception as e:
        #print(e)
        #print('Not opened', img_url)
        pass

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [49]:
classes = [
 'Education', 
 'Entertainment',
 'Beauty',
 'Bank',
 'Fitness',
 'Food',
 'Health',
 'housework',
 'Religious',
 'Travel']

In [50]:
classes = [cls.lower() for cls in classes]
classes

['education',
 'entertainment',
 'beauty',
 'bank',
 'fitness',
 'food',
 'health',
 'housework',
 'religious',
 'travel']

In [18]:
word2vec = {}
with open(os.path.join(FOLDER_PATH, 'data', 'glove_data', 'glove.6B.300d.txt')) as file:
    for line in file:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vector

In [19]:
model = VGG16()

In [62]:
img_labels = []
for image_file in os.listdir(os.path.join(FOLDER_PATH, 'advertiser-quality-from-sites', 'source', 'images')):
    image = load_img(os.path.join('images', image_file), target_size=(W, H))
    image = img_to_array(image)
    image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
    image = preprocess_input(image)
    y_pred = model.predict(image)
    label = decode_predictions(y_pred, top=3)
    cls_name = label[0][0][1]
    cls_name = re.sub('[^a-zA-Z]', ' ', cls_name)
    img_labels.append(cls_name.split())

In [72]:
for _, label in zip(range(10), img_labels):
#    label_word2vec = []
#     for i in range(0, len(label)):
#         if not label[i] in word2vec: continue
#         if len(label_word2vec) == 0: 
#             label_word2vec = word2vec[label[i]]
#         else:
#             label_word2vec = [a + b for a, b in zip(label_word2vec, word2vec[label[i]])]
    label_word2vec = []
    for i in range(0, len(label)):
        if not label[i] in word2vec: continue
        label_word2vec.append(word2vec[label[i]])
    if len(label_word2vec) == 0:
        print('The label is not in dictionary')
        continue
    label_word2vec = np.mean(label_word2vec, axis=0)
    dist = []
    for cls in classes:
        dist.append(distance.cosine(label_word2vec, word2vec[cls]))
    best_idx = np.argmin(dist)
    print(label, 'is closest to', classes[best_idx], 'with distance:', dist[best_idx])

['rugby', 'ball'] is closest to fitness with distance: 0.7197335362434387
['sunscreen'] is closest to housework with distance: 0.8552117943763733
['book', 'jacket'] is closest to travel with distance: 0.7574307471513748
['face', 'powder'] is closest to food with distance: 0.7368230223655701
['gondola'] is closest to travel with distance: 0.8398619294166565
['chime'] is closest to housework with distance: 0.9377572648227215
['lotion'] is closest to beauty with distance: 0.8354157656431198
['gondola'] is closest to travel with distance: 0.8398619294166565
['microphone'] is closest to entertainment with distance: 0.9198270589113235
['plate'] is closest to food with distance: 0.8805762901902199


In [ ]:
i = 0
for img_url in img_urls:
    try:
        r = requests.get(img_url, stream=True, timeout=3)
        file_name = 'images/pic{}.jpg'.format(i)
#         #print(img_url)
#         with open('images/pic{}.jpg'.format(i), 'wb') as f:
#             f.write(r.content)
        img = Image.open(BytesIO(r.content))
        if img.mode in ['RGBA', 'P']:
            img = img.convert('RGB')
        img.save(file_name, 'JPEG')
        
        i += 1
    except Exception as e:
        print(e)
        #print('Not opened', img_url)
        pass

In [ ]:
images_rgb = []
for x in range(200):
    file = 'images/pic{}.jpg'.format(x)
    img = matimg.imread(file)
    #print(file + ':', img.shape)
    images_rgb.append(img)

In [ ]:
H, W = 224, 224
for img_rgb in images_rgb:
    mat = tf.constant(img_rgb)
    mat = tf.image.resize_with_crop_or_pad(mat, H, W)
    mat /= 255.0

In [ ]:
model = VGG16()